# **TUGAS 1 - Scraping MetaData Putusan**

Dalam Tugas 1 ini atau Tahap 1 ini bertujuan untuk melakukan proses yang dinamakan scraping metadata putusan pengadilan negeri Banyuwangi, berikut linknya https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-banyuwangi/kategori/pidana-umum-1

Dibuat oleh :

*   Nama    : Abdul Rahem Faqih
*   Nim     : 220411100029
*   Kelas   : Ekstraksi Informasi A


## **STEP 1 - Mendapatkan URL putusan**
Mendapatkan setiap url pada link halaman di atas, 1 page mempunyai 20 putusan, disini saya mengambil page **51 - 107**, jadi total link yang terambil adalah **56 x 20 = 1.120** link putusan yang akan disimpan di file txt.

### Import Library yang diperlukan

In [ ]:
import requests
import time
import warnings
from bs4 import BeautifulSoup
import re
import csv
import os.path




*   **request** digunakan untuk melakukan permintaan HTTP untuk mengakses konten dari website
*   **time**  digunakan untuk menghitung eksekusi dari proses program
*   **warnnings** digunakan untuk menonaktifkan peringatan

*   **BeautifulSoup** dari bs4  digunakan untuk memparsing html dan mengambil elemen elemen bahkan text dari elemen itu dari halaman website
*   **re** digunakan untuk pencocokan pola menggunakan regular expression
*   **csv** untuk menulis dan menyimpan data dalam format csv
*   **os**  digunakan ketika ingin berinteraksi dengan sistem operasi seperti membuat direktori


### Kode untuk mengambil link putusan

In [ ]:
def getURLfromWeb(url):

    response = requests.get(url, verify=False)
    print(response)

    htmlCode1 = BeautifulSoup(response.text, 'html.parser')
    result1 = htmlCode1.findAll('a')

    urlHasil=[] # tempat untuk menyimpan kumpulan url perhalaman
    for eachResult1 in result1:
        cariURLawal=str(eachResult1).find('https://putusan3.mahkamahagung.go.id/direktori/putusan/')
        cariURLakhir=str(eachResult1).find('html">Putusan') + 4
        if cariURLawal == 9 and cariURLakhir >= 4:
            cariURL=str(eachResult1)[cariURLawal:cariURLakhir]
            urlHasil.append(cariURL)
    return urlHasil

In [ ]:
def main():

    warnings.filterwarnings('ignore')

    url = "https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-banyuwangi/kategori/pidana-umum-1"


    end = 108 + 1
    listHasil = []
    start = 51


    startTime = time.time()
    file_hasil = open("/content/drive/MyDrive/Colab Notebooks/INFORMATION EXTRACTION/hasilListURLPage.txt", "w", encoding='UTF8')

    for i in range(start, end):
        url = f"{url}/page/{i}.html"
        listHasil = getURLfromWeb(url)
        print(f"--- PAGE {i}")

        for listURL in listHasil:
            file_hasil.write(listURL+"\n")

    file_hasil.close()
    endTime = time.time()
    print(listHasil)
    print('Time Processing : ', endTime-startTime, ' Second')

main()

pada fungsi **getURLfromWeb** digunakan untuk melakukan proses pengambilan url putusan yang dinginkan pada halaman putusan pengadilan negeri banyuwangi

![gambar putusan.png](https://github.com/abdulrahemfaqih/kumpulan_gambar/blob/main/IE/gambar%20putusan.png?raw=true)

gambar di atas ialah salah satu link yang akan di ambil, fungsi di atas akan mengambil setiap link dari page 51 sampai page 108


## **STEP 2 - Mendapatkan MetaData Putusan**

Pada step 2 ini nertujuan untuk mendapatkan metaData dari url yang kita sudah dapatkan, berikut contoh halaman yang metadanya ingin kita ambil https://putusan3.mahkamahagung.go.id/direktori/putusan/zaec82579b6441609a3f313233353332.html. yang ingin kita ambil yaitu terdakwa, penuntut umum, sampai link pdf putusannya. Semua akan disimpan ke dalam format csv yang akan dijadikan dataset

### Kode untuk generate Meta (mengambil Metadata)

In [ ]:
def generateMeta(urlMeta):

    url = str(urlMeta).strip()
    response = requests.get(url, verify=False)
    print(response)

    soup = BeautifulSoup(response.text, 'html.parser')
    cleanTags = re.compile('<.*?>')

    # list data yang akan di ambil
    listMetaHead = ["terdakwa", "penuntut_umum", "nomor", "tingkat_proses", "klasifikasi", "kata_kunci", "tahun", "tanggal_register",
              "lembaga_peradilan", "jenis_lembaga_peradilan", "hakim_ketua", "hakim_anggota", "panitera", "amar",
              "amar_lainnya", "catatan_amar", "tanggal_musyawarah", "tanggal_dibacakan", "kaidah", "abstrak", "url"]

    listMeta = []

    rowsMETA1 = soup.find("ul", {"class": "portfolio-meta nobottommargin"}).find("table").findAll("tr")
    rowsMETA2 = soup.findAll("ul", {"class": "portfolio-meta nobottommargin"})

    #print('-------------------------')
    for row in rowsMETA1:
        coll = row.findAll("td")

        cleantext2 =''
        cleantext1 =''

        if len(coll) > 1:
            cleantext2 = (re.sub(cleanTags, '', str(coll[1]))).strip()
            listMeta.append(cleantext2.replace('\n',' '))
        else:
            cleantext1 = (re.sub(cleanTags, ' ', str(coll[0]))).strip()
            # untuk putusan pidana akan muncul terdakwa dan penuntut umum pada meta
            # sedangkan untuk putusan selain pidana tidak muncul

            pidorpdt = re.search( r'(.*)/Pdt.(.*)',str(cleantext1), re.M|re.I)

            # check dokumen putusannya pidana atau perdata
            if pidorpdt == None:
                entTerdakwah = re.search( r'(.*)Terdakwa:(.*)',str(cleantext1), re.M|re.I)
                entPenuntut = re.search( r'Penuntut Umum:(.*)Terdakwa:',str(cleantext1), re.M|re.I)
            else:
                entTerdakwah = re.search( r'(.*)Tergugat:(.*)',str(cleantext1), re.M|re.I)
                entPenuntut = re.search( r'Penggugat:(.*)Tergugat:',str(cleantext1), re.M|re.I)

            if entTerdakwah == None:
                listMeta.append("")
            else:
                listMeta.append(entTerdakwah.group(2))

            if entPenuntut == None:
                listMeta.append("")
            else:
                listMeta.append(entPenuntut.group(1))
    # proses url dokumen
    urlDL = rowsMETA2[1].findAll("li")
    urlDLStr = str(urlDL[4])
    listMeta.append(urlDLStr[urlDLStr.find("https"):urlDLStr.find('">')])

    return listMeta

Fungsi `generateMeta()` bertujuan untuk mengambil metadata dari sebuah halaman web dengan melakukan permintaan HTTP GET ke URL yang diberikan, kemudian memparsing teks HTML respons menggunakan BeautifulSoup. Fungsi ini mengekstrak berbagai metadata terkait putusan pengadilan, seperti nama terdakwa, penuntut umum, nomor kasus, tingkat proses, dan URL dokumen. Metadata yang diambil berasal dari elemen-elemen HTML tertentu, yang diproses menggunakan teknik pencarian tag HTML dan regex untuk membedakan apakah dokumen tersebut adalah putusan pidana atau perdata. Jika regex menemukan data yang sesuai, seperti nama terdakwa atau tergugat, data tersebut akan ditambahkan ke dalam daftar metadata. Selain itu, fungsi ini juga mengekstrak URL dokumen dari halaman. Hasil akhirnya adalah daftar metadata yang berisi informasi penting yang telah diekstrak dan dibersihkan dari halaman web tersebut.

### Kode untuk generate kedalam format csv

Setelah dapat metadatanya nah informasi tersebut akan di simpan ke format csv

In [ ]:
def generateFileCSV(listHasil,csvName1):

	csvName = csvName1 #nama file csv

	if os.path.exists(csvName): # cek ada atau belum
		# jika ada maka akan menambahkan
		f = open(csvName, 'a', newline='\n')
		print(f)
		print("ada")
		w = csv.writer(f)

	else:
		# jika tidaka da maka akan membuat baru
		f = open(csvName, 'w', newline='\n')
		print(f)
		print("tidak ada")
		# menambahkan baris pertama (header)
		w = csv.writer(f)
		w.writerow(("terdakwa", "penuntut_umum", "nomor", "tingkat_proses", "klasifikasi", "kata_kunci", "tahun", "tanggal_register",
              "lembaga_peradilan", "jenis_lembaga_peradilan", "hakim_ketua", "hakim_anggota", "panitera", "amar",
              "amar_lainnya", "catatan_amar", "tanggal_musyawarah", "tanggal_dibacakan", "kaidah", "abstrak", "url"))

	# menulis file csv
	for s in listHasil:
		# mengiterasi data list hasil satu persatu lalu dimasukkan seusai dengan kolomnya
		w.writerow(s)
	#tutup file
	f.close()
	berhasil = "\nCreate Csv file Berhasil\n"
	return berhasil

Fungsi `generateFileCSV()` bertujuan untuk membuat atau menambahkan data ke dalam file CSV. Fungsi ini menerima dua parameter: `listHasil`, yaitu daftar data yang akan dimasukkan ke dalam file CSV, dan `csvName1`, yang merupakan nama file CSV. Pertama, fungsi memeriksa apakah file dengan nama yang diberikan sudah ada menggunakan `os.path.exists()`. Jika file sudah ada, fungsi membuka file tersebut dalam mode append untuk menambahkan data baru ke bagian akhir file. Jika file belum ada, fungsi membuat file baru dalam mode write dan menambahkan header berisi nama-nama kolom yang sesuai seperti `terdakwa`, `penuntut_umum`, `nomor`, dan sebagainya. Setelah itu, fungsi menulis setiap elemen dari listHasil ke file CSV baris demi baris. Terakhir, file CSV ditutup untuk memastikan data tersimpan dengan benar, dan fungsi mengembalikan pesan bahwa pembuatan atau penambahan data ke file CSV telah berhasil.

### Kode utama untuk mengambil metada dari pagenya dan menyimpan ke format csv

In [ ]:
def main():

    warnings.filterwarnings('ignore')

    fileListURL = "/content/drive/MyDrive/Colab Notebooks/INFORMATION EXTRACTION/hasilListURLPage.txt"
    fileMetaCSV = "/content/drive/MyDrive/Colab Notebooks/INFORMATION EXTRACTION/metaPidananUmumPN-Banyuwangi-Page-57-107.csv"
    listHasil =[]

    startTime = time.time()
    # membaca setiap link pada file txt
    openfileListURL = open(fileListURL, "r", encoding='UTF8')
    bacaListURL = openfileListURL.readlines()

    ukuran_batch = 200

    i = 1
    for barisURL in bacaListURL:
        try:
            # ekstrak informasi metadata
            hasil = generateMeta(str(barisURL))
            listHasil.append(hasil)
            print(f"link ke - {i} berhasil diproses.")

            if len(listHasil) >= ukuran_batch:
                generateFileCSV(listHasil, fileMetaCSV)
                listHasil = []
        except Exception as e:
            print(f"Error Get Meta Inf, {e}")
        i=i+1

    createFile = generateFileCSV(listHasil,fileMetaCSV)

    openfileListURL.close()
    endTime = time.time()
    print('Time Processing : ', endTime-startTime, ' Second')

main()

Fungsi `main()` berfungsi sebagai program utama yang mengatur proses pengambilan dan penyimpanan metadata dari beberapa halaman web dalam bentuk CSV. Pertama, fungsi ini menonaktifkan peringatan menggunakan `warnings.filterwarnings('ignore')` dan menetapkan dua variabel: `fileListURL`, yang berisi path ke file teks dengan daftar URL yang akan diekstrak, serta `fileMetaCSV`, yaitu path di mana hasil metadata akan disimpan dalam file CSV. Data metadata yang dihasilkan akan disimpan sementara dalam `listHasil`. Fungsi ini kemudian mencatat waktu mulai pemrosesan dengan `time.time()`.

Setelah itu, fungsi membuka dan membaca file teks `fileListURL` yang berisi daftar URL. Setiap baris di file tersebut berisi URL yang akan diproses untuk mengekstraksi metadata menggunakan fungsi `generateMeta()` yang telah dijelaskan sebelumnya. Fungsi `generateMeta()` digunakan di dalam loop yang mengiterasi setiap URL. Metadata dari setiap URL yang berhasil diproses ditambahkan ke dalam `listHasil`, dan setiap kali jumlah hasil mencapai batas ukuran batch yang ditentukan (`ukuran_batch = 200`), fungsi `generateFileCSV()` dipanggil untuk menyimpan metadata yang terkumpul ke file CSV. Fungsi ini bertugas menulis data ke file CSV, baik dengan membuat file baru atau menambahkannya jika sudah ada, seperti yang dijelaskan sebelumnya.

Jika terjadi kesalahan saat memproses URL tertentu, program menangkap pengecualian menggunakan blok `try-except`, dan melanjutkan ke URL berikutnya, sambil mencatat kesalahan. Setelah semua URL diproses, sisa metadata yang belum tersimpan dalam batch terakhir akan disimpan ke file CSV menggunakan `generateFileCSV()`. File yang dibuka untuk membaca URL kemudian ditutup, dan waktu total yang dibutuhkan untuk memproses semua URL dihitung dan ditampilkan.

Secara keseluruhan, fungsi `main()` mengatur alur kerja utama dengan memanfaatkan fungsi `generateMeta()` untuk ekstraksi metadata dari setiap URL dan fungsi `generateFileCSV()` untuk menyimpan hasil tersebut ke file CSV dalam batch. Kedua fungsi ini bekerja bersama-sama untuk memastikan proses pengekstrakan metadata dari sejumlah URL berjalan dengan efisien dan data hasilnya tersimpan dengan benar.


## **STEP 3 - Prepocessing Dataset MetaData**

Pada taham ini bertujuan untuk memberishkan dataset yang sudah kita dpatkan, yang nantinya kita akan menghapus baris data yang mempunyai missing vale pada kolom url pdfnya, karena pada tahep selanjutnya ialah kita mengunduh file pdf yang ada pada kolom url pdfnya

### Import Library yang digunakan

In [ ]:
import pandas as pd

### Load Dataset

In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/INFORMATION EXTRACTION/metaPidananUmumPN-Banyuwangi-Page-57-107.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')
data.head()

,terdakwa,penuntut_umum,nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,...,hakim_anggota,panitera,amar,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,kaidah,abstrak,url
0,ABDUL ASIS Als. AZIS Bin Alm TACE,"1.SUPRIYADI AHMAD, SH.,M.H 2.VIRDIS FIRMANILL...",90/Pid.Sus/2022/PN Byw,Pertama,Pidana Umum,Perlindungan Anak,2022,7 Februari 2022,PN BANYUWANGI,PN,...,"Hakim Anggota Philip Pangalila, Br Hakim Angg...",Panitera Pengganti: Andi Setyawan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,MENGADILI Menyatakan Terdakwa Abdul Asis Alias...,17 Mei 2022,17 Mei 2022,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...
1,M ARIF JULIAWAN,NaN,15/Pid.C/2022/PN Byw,Pertama,Pidana Umum,Pelanggaran,2022,17 Mei 2022,PN BANYUWANGI,PN,...,Hakim Tunggal Firlando,Panitera Pengganti: M. Tofik Djulianto,Lain-lain,PIDANA DENDA,"MENGADILI : Menyatakan, bahwa Terdakwa M.Arif ...",17 Mei 2022,17 Mei 2022,—,NaN,NaN
2,Paijem,NaN,16/Pid.C/2022/PN Byw,Pertama,Pidana Umum,Pelanggaran,2022,19 Mei 2022,PN BANYUWANGI,PN,...,Hakim Tunggal Yustisiana,Panitera Pengganti: Andi Setyawan,Lain-lain,PIDANA KURUNGAN,MENGADILI : Menyatakan Terdakwa Paijem secara ...,19 Mei 2022,19 Mei 2022,—,NaN,NaN
3,PONIYEM,NaN,17/Pid.C/2022/PN Byw,Pertama,Pidana Umum,Pelanggaran,2022,19 Mei 2022,PN BANYUWANGI,PN,...,Hakim Tunggal Yustisiana,Panitera Pengganti: Andi Setyawan,Lain-lain,PIDANA KURUNGAN,MENGADILI : Menyatakan Terdakwa Poniyem secara...,19 Mei 2022,19 Mei 2022,—,NaN,NaN
4,SUYATI,NaN,18/Pid.C/2022/PN Byw,Pertama,Pidana Umum,Pelanggaran,2022,19 Mei 2022,PN BANYUWANGI,PN,...,Hakim Tunggal Yustisiana,Panitera Pengganti: Andi Setyawan,Lain-lain,PIDANA KURUNGAN,MENGADILI : Menyatakan Terdakwa Suyati secara ...,19 Mei 2022,19 Mei 2022,—,NaN,NaN


terdapat 1098 baris dan 21 kolom

## Info Data

In [ ]:
print(data.shape)

(1098, 21)


In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098 entries, 0 to 1097
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   terdakwa                 1096 non-null   object 
 1   penuntut_umum            744 non-null    object 
 2   nomor                    1098 non-null   object 
 3   tingkat_proses           1098 non-null   object 
 4   klasifikasi              1098 non-null   object 
 5   kata_kunci               1096 non-null   object 
 6   tahun                    1098 non-null   int64  
 7   tanggal_register         1098 non-null   object 
 8   lembaga_peradilan        1098 non-null   object 
 9   jenis_lembaga_peradilan  1098 non-null   object 
 10  hakim_ketua              1098 non-null   object 
 11  hakim_anggota            1098 non-null   object 
 12  panitera                 1098 non-null   object 
 13  amar                     1098 non-null   object 
 14  amar_lainnya            

### Misiing Value

In [ ]:
print(data.isnull().sum())

terdakwa                      2
penuntut_umum               354
nomor                         0
tingkat_proses                0
klasifikasi                   0
kata_kunci                    2
tahun                         0
tanggal_register              0
lembaga_peradilan             0
jenis_lembaga_peradilan       0
hakim_ketua                   0
hakim_anggota                 0
panitera                      0
amar                          0
amar_lainnya                  0
catatan_amar                  0
tanggal_musyawarah            0
tanggal_dibacakan             0
kaidah                        0
abstrak                    1098
url                         715
dtype: int64


### Hapus baris data yang memiliki missing value pada **kolom url** dan asimpan hasilnya

In [ ]:
data_cleaned = data.dropna(subset=['url'])
print(f"Jumlah baris data setelah dihapus penghapusan: {data_cleaned.shape[0]} baris")
output_path = "/content/drive/MyDrive/Colab Notebooks/INFORMATION EXTRACTION/cleaned_metaPidananUmumPN-Banyuwangi-Page-57-107.csv"
data_cleaned.to_csv(output_path, index=False)

Jumlah baris data setelah dihapus penghapusan: 383 baris


## **STEP 4 - Download file pdf putusan**

pada step ini bertujuan mendownload file pdf yang ada di kolom url pada dataset yang sudah di bersihkan

In [ ]:
def download_files_from_csv(csv_file_path, output_folder):
    with open(csv_file_path, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # skip header
        for row in csv_reader:
            url = row[20]
            download_file(url, output_folder)



def download_file(url, output_folder):
    response = requests.get(url)
    if response.status_code == 200:
        file_name = url.split("/")[-1]
        file_path = f"{output_folder}/{file_name}.pdf"
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {url}")

if __name__ == "__main__":
    csv_file_path = "/content/drive/MyDrive/Colab Notebooks/INFORMATION EXTRACTION/cleaned_metaPidananUmumPN-Banyuwangi-Page-57-107.csv"
    output_folder = "/content/drive/MyDrive/Colab Notebooks/INFORMATION EXTRACTION/PDF_PUTUSAN"
    download_files_from_csv(csv_file_path, output_folder)

Downloaded: zaecda3d6485d6ac98be303930393335
Downloaded: zaeccfff145df44a87e8303831383230
Downloaded: zaecb0c4b516c434b9d4313433323233
Downloaded: zaec617274f1a0dc85fd313535343335
Downloaded: zaec4618d1e9e544bf32323033343535
Downloaded: zaec403930c02d0ca562303931313331
Downloaded: zaec1533158a52c8b57e313530393239
Downloaded: zaec1533fffe5c389678313530383532
Downloaded: zaec1532f7412882a4c7313530383338
Downloaded: zaec1532f640e12abed8313530383336
Downloaded: zaec1532c20019bcbfc7313530373038
Downloaded: zaec1532c1027690a4e2313530373037
Downloaded: zaec1532bd338a408fd9313530373030
Downloaded: zaec1532b8d849eaa413313530363533
Downloaded: zaec1532a902ba8c96d3313530363236
Downloaded: zaebc7940f34f690987d323032373038
Downloaded: zaebc7940d0ec9689c15323032373035
Downloaded: zaebc7940be8249e9473323032373033
Downloaded: zaebc79401fb6a728b4b323032363436
Downloaded: zaebc793f95aff7ca613323032363332
Downloaded: zaebc33e9e1cc310adf5303830353237
Downloaded: zaebc33e7a61dbcc89b0303830343237
Downloaded

Setelah file CSV yang berisi metadata berhasil dibuat menggunakan fungsi sebelumnya dan dibersihkan, kode ini digunakan untuk mendownload file PDF berdasarkan URL yang terdapat di kolom ke-20 dari file CSV tersebut. Fungsi utama, `download_files_from_csv()`, membuka dan membaca file CSV, kemudian mengambil URL dari kolom ke-20 (indeks ke-19) di setiap baris data. URL tersebut digunakan untuk memanggil fungsi `download_file()`, yang bertanggung jawab untuk mendownload file PDF dari URL yang diberikan dan menyimpannya di folder output yang telah ditentukan. `Fungsi download_file()` memeriksa apakah permintaan download berhasil `(status code 200)`, lalu menyimpan file dengan nama yang diambil dari URL dan menambahkan ekstensi .pdf.